# Monty Hall Problem

The Monty Hall Problem (https://en.wikipedia.org/wiki/Monty_Hall_problem) is a very famous problem in Probability Theory. It can be described as follow:

Suppose that you are on a game-show where you can win a car if you find behind which door it is. There are three closed doors ("\{0,1,2\}") and only behind one of them there is a car; behind the others, there is a goat. 

<img src="images/Monty_open_door.png" style="width:300px"/>

You have two opportunities to pick a door. Firstly, you have to pick one among all three doors. Say you pick Door #1, and the Host, the person in charge who knows where the car is, opens **another** door, say Door #0, which has a goat. Then, you are given a second opportunity to choose again. The Host asks to you: "Do you want to keep Door #1 or would you prefer changing to Door #2?"

According to the available information, should the Player change her mind?

<h2>A PGM's answer:</h2>

Let us model this problem by means of a Bayesian network:

In [ ]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
import numpy as np

We can identify 3 different random variables:

- Prize position: $ P \in \{0, 1, 2 \} $

- Player's Answer: $ A \in \{0, 1, 2\} $

- Host's choice: $ H \in \{0, 1, 2\} $

where each possible value in $P$, $A$ and $H$ identifies a door. 


Note that the Host makes his decision when the Player already said her first choice (and knowing where the car is). Thus, his choice depends on both the Prize position and the Player's choice:

<img src="images/bn_monty_hall.png"/>



We can safely assume that the Prize has been put behind a door without any specific criteria. Therefore, the marginal probability distribution of P is uniform:


In [ ]:
cpd_p = TabularCPD('P', variable_card=3, values=np.array([[0.33], [0.33], [0.33]]))


The player, without any knowledge, is probably making her first Answer randomly. Thus, we can assume that the marginal probability distribution of A is uniform:


In [ ]:
cpd_a = TabularCPD('A', variable_card=3, values=([ #### YOUR CODE HERE ####])

The probability distribution of the remaining variable, the one modeling the behavior of the Host can be inferred from the structure of the problem. The Host chooses a door different from the player's Answer (specifically one which does not have the prize) and opens it. With this information, one can complete the table of probabilities (parameters) of this CPD:

<pre>
+---+---+---+----------+
| A | P | H | p(H|A,P) |
+---+---+---+----------+
| 0 | 0 | 0 | 0.0      | The Host won't never open the door selected by the player; in this case, Door #0
| 0 | 0 | 1 | 0.5      | If the Prize is behind player's door (A=P), the Host can open any other door
| 0 | 0 | 2 | 0.5      |
+---+---+---+----------+
| 0 | 1 | 0 | 0.0      |
| 0 | 1 | 1 | 0.0      |
| 0 | 1 | 2 | 1.0      | If the Prize is at Door #1 and the player Answers Door #0, the Host can only 
+---+---+---+----------+ open Door #2 without revealing the location of the Prize.
| 0 | 2 | 0 | 0.0      |
| 0 | 2 | 1 | 1.0      | Same here
| 0 | 2 | 2 | 0.0      |
+---+---+---+----------+
| A | P | H | p(H|A,P) |
+---+---+---+----------+
| 1 | 0 | 0 | 0.0      |
| 1 | 0 | 1 | 0.0      |
| 1 | 0 | 2 | 1.0      | Same here
+---+---+---+----------+
| 1 | 1 | 0 | 0.5      | As the Prize is behind player's door (A=P), the Host can open any other door
| 1 | 1 | 1 | 0.0      |
| 1 | 1 | 2 | 0.5      |
+---+---+---+----------+
| 1 | 2 | 0 | 1.0      | The Host can only open Door #0 without revealing the location of the Prize
| 1 | 2 | 1 | 0.0      |
| 1 | 2 | 2 | 0.0      |
+---+---+---+----------+
| A | P | H | p(H|A,P) | COMPLETE THIS LAST PART
+---+---+---+----------+
| 2 | 0 | 0 |          |
| 2 | 0 | 1 |          | 
| 2 | 0 | 2 |          |
+---+---+---+----------+
| 2 | 1 | 0 |          |
| 2 | 1 | 1 |          |
| 2 | 1 | 2 |          |
+---+---+---+----------+
| 2 | 2 | 0 |          |
| 2 | 2 | 1 |          |
| 2 | 2 | 2 |          |
+---+---+---+----------+
</pre>

In [ ]:
# Defining the CPDs:
cpd_h = TabularCPD('H', variable_card=3, values=[[0.0, 0.0, 0.0, 0.0, 0.5, 1.0, ### YOUR ### , , ], 
                                                 [0.5, 0.0, 1.0, 0.0, 0.0, 0.0, ### CODE ### , , ], 
                                                 [0.5, 1.0, 0.0, 1.0, 0.5, 0.0, ### HERE ### , , ]],
                  evidence=['A', 'P'], evidence_card=[3, 3])


Now, we just combine all three CPDs to form a Bayesian network model:

In [ ]:
# Defining the network structure
model = BayesianModel([('A', 'H'),  # A is a parent of H,
                       ('P', 'H')]) # P is a parent of H
# Associating the CPDs with the network structure
model.add_cpds(cpd_a, cpd_p, cpd_h)


Once the BN is built, we can check if everything is ok:


In [ ]:
# Some other methods
print(model.get_cpds())

# check the model structure and the associated CPD 
print("Is this a proper BN?",model.check_model())


Now we have modeled our world. Let's see what happens when we put it to work!


<hr/>
Let's say that the player initially Answers 'Door #0' and, in turn, the Host opens Door #2. Basic intuition probably says that the player has now a choice among two possible options, and both are equally likely to hide the prize. But, **<b>is this intuition right</b>**? 

We can use our model to calculate the probability distribution of the Prize's location in this specific case, i.e., $P(P | A=0, H=2)$. To do so, we need to use inference techniques. Variable elimination is one of the simplest algorithms for making inference:


In [ ]:
# Infering the posterior probability 
from pgmpy.inference import VariableElimination

infer = VariableElimination(model)

cpq = infer.query(['P'], evidence={'A': 0, 'H': 2}) # Player's Answer = 0; Host reply = 2

print(cpq)

Note that the probability of having the prize behind Door #1 is larger than behind Door #0. In this problem, if we take into account all the available information (including that given when the Host selects a specific Door), we can see that the best possible action is to change the initially selected door.

<hr>

Now, try yourself! Change the values of $A$ and $H$ (with the only condition that they cannot be the same). You will realize that the same conclusion holds for all the possible cases!